>## 1. Análise de Sentimentos

### 1. Objetivo
O objetivo da análise de sentimentos e classificação de textos é determinar o valor subjetivo de um documento de texto. Aqui trabalharemos apenas com um classificaçao de polaridade em extremos, ou seja, classificar como positivo ou negativo o conteúdo de um documento de texto.
#### 1. Problema
O problema proposto para abordar a aprendizagem da análise de sentimentos é descrito a seguir:

A empresa Amazon deseja obter um sistema inteligente para processar os comentários
de seus clientes sobre os seus produtos, podendo classificar tais comentários dentre as
categorias: positivo ou negativo. Para isso ela disponibiliza três bases de dados com
sentenças rotuladas.
#### 1. Os Dados
Os dados estão organizados em sentença e rótulo, sendo 0 negativo e 1 positivo
As bases são provenientes dos seguintes sites:
* imdb.com
* amazon.com
* yelp.com


### 1. Preparação

Alguns módulos e bibliotecas devem ser instalados para a execução deste projeto. Para adiantar essa fase de preparação e concluí-la rapidamente sem mais preocupações, disponibiliza-se todo o ambiente utilizando ferramenta `Docker`. O Dockerfile pode ser visualizado abaixo:

```Dockerfile
FROM python:3 
ENV PYTHONUNBUFFERED 1 

RUN mkdir /code 
WORKDIR /code 

ADD . /code/
RUN pip install scikit-learn pandas matplotlib scipy jupyter nltk

RUN chmod +x boot.sh

EXPOSE 8888
CMD ["/bin/sh", "./boot.sh"]
```

Junto com o Dockerfile, apresenta-se também o arquivo boot.sh, que instala alguns módulos da biblioteca `nltk` e inicia a execução do `jupyter notebook`.
```
python << END
import sys
import nltk

nltk.download('punkt')
nltk.download('stopwords')
END

jupyter notebook --ip=0.0.0.0 --allow-root
```

Para utilizar isso, primeiro installe o docker e em seguida execute o seguinte comando no diretório do seu Dockerfile:
```
docker build -t machine-learn .
```

Para executar o container, use o comando a seguir substituindo o `~` pelo path onde fez o clone do repositório:
```
docker run --name machine-learn-container -p 8888:8888 -v ~/amazon/jeferson:/code machine-learn:latest /bin/sh ./boot.sh
```

### 1. Pré-processamento

As três bases de dados do problema devem ser usadas em conjunto. A biblioteca pandas pode ser usada para facilitar esse processo. O código abaixo importa as bases e as concatena em uma base maior. Ao final, é gerado um arquivo com extensão `csv`, o qual será usado para os treinos e análises futuras.

In [ ]:
import pandas

imdb = pandas.read_csv('data/imdb_labelled.txt', sep="\t", names=["sentences", "polarity"])
yelp = pandas.read_csv('data/yelp_labelled.txt', sep="\t", names=["sentences", "polarity"])
amazon = pandas.read_csv('data/amazon_cells_labelled.txt', sep="\t", names=["sentences", "polarity"])

big = pandas.DataFrame()
big = big.append([imdb, yelp, amazon])

big.to_csv('big.csv', index=False, encoding='utf-8')

### 1. Tokenization

Para a criação de um dicionário, precisa-se transformar o texto em tokens. Para auxiliar nessa tarefa pode-se utilizar uma biblioteca para processamento de linguagem natural, no exemplo abaixo, usa-se o `nltk`. O `nltk` é uma biblioteca Python de código aberto que realiza esta função.

In [ ]:
import nltk

sentence = 'My test for nltk library!!'
tokens = nltk.word_tokenize(sentence)

print(tokens)

### 1. Dicionário

Para criar um dicionário, precisamos usar apenas a coluna que contém as sentenças no arquivo e ignorar a coluna da polaridade. Novamente, o `nltk` facilita esse processo. Obtidas as sentenças, usaremos o `nltk` para quebrá-las em tokens.

In [ ]:
import nltk

sentences = big['sentences']
all_sentences_strigs = sentences.str.lower()

all_sentences_tokenized = [] #Has 
for sentence_string in all_sentences_strigs:
    sentence_tokenized = nltk.word_tokenize(sentence_string)
    all_sentences_tokenized.append(sentence_tokenized)

all_tokens = [] # Has all sentences tokens
for sentence_tokenized in all_sentences_tokenized:
    all_tokens.extend(sentence_tokenized)
    
dictionary = set()
dictionary.update(all_tokens)

### 1. Word normalization

Uma abordagem comum na análise de sentimentos é o uso de digramas ou trigramas, isso auxilia na classificação de sentenças. Dado um vetor de tokens onde já se foram eliminados as repetições e stop words, podemos criar digramas como segue:

In [ ]:
dictionary_of_digrams = set()

tokens_for_test = ['teste1', 'teste2', 'teste3', 'teste4']

for x in range(len(tokens_for_test)):
    if x + 1 < len(tokens_for_test):
        digram = [tokens[x]+' '+tokens_for_test[x+1]]
        dictionary_of_digrams.update(digram)
        
print(dictionary_of_digrams)

### 1.  Vetorização
Usa-se a vetorização para tornar possível o trabalho dos classificadores. Na vetorização do texto, cada palavra é tratada como uma característica

In [ ]:
all_sentences_vetorized = []
for sentence_tokenized in all_sentences_tokenized:
    for token in sentence_tokenized:
        if token in all_tokens_indexed:
            tonken_position = all_tokens_indexed[token]
            sentence_tokenized[tonken_position] += 1
        all_sentences_vetorized.append(sentence_tokenized)
        
X = numpy.array(all_sentences_vetorized)